In [12]:
import pandas as pd
import numpy as np
import pickle
import warnings
import seaborn as sns
import scipy.stats as stats
from collections import Counter
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)

In [13]:
d_league = {"ENG":{'id':17,'season':{"22-23":41886}},
             "GER":{'id':35,'season':{"22-23":42268}},
             "SPA":{'id':8,'season':{"22-23":42409}},
             "ITA":{'id':23,'season':{"22-23":42415}},
             "FRA":{'id':34,'season':{"22-23":42273}}}

In [14]:
# Initialize list for match stats
l_match_stats = []

# Loop through each league and season in the provided dictionary
for league, item in d_league.items():
    print(f"Processing league: {league}")
    for season, season_id in item['season'].items():
        # Only process the "22-23" season
        if season == "22-23":
            # Load match data from pickle
            with open(f'/content/{league}-{season}.pickle', 'rb') as handle:
                d_matches = pickle.load(handle)

            # Process each match in the loaded data
            for id, info in d_matches.items():
                # Only consider matches that have statistics and are completed
                if 'statistics' in info.keys() and info['info']['status']['code'] != 70:
                    d_home_stats = {}
                    d_away_stats = {}

                    # Populate home team stats
                    d_home_stats['league'] = league
                    d_home_stats['season'] = season
                    d_home_stats['round'] = info['info']['roundInfo']['round']
                    d_home_stats['matchId'] = id
                    d_home_stats['teamId'] = info['info']['homeTeam']['id']
                    d_home_stats['teamName'] = info['info']['homeTeam']['shortName']
                    d_home_stats['teamLongName'] = info['info']['homeTeam']['name']
                    d_home_stats['formation'] = info['lineups']['home']['formation']
                    d_home_stats['isHome'] = True
                    d_home_stats['opponentId'] = info['info']['awayTeam']['id']

                    # Populate away team stats
                    d_away_stats['league'] = league
                    d_away_stats['season'] = season
                    d_away_stats['round'] = info['info']['roundInfo']['round']
                    d_away_stats['matchId'] = id
                    d_away_stats['teamId'] = info['info']['awayTeam']['id']
                    d_away_stats['teamName'] = info['info']['awayTeam']['shortName']
                    d_away_stats['teamLongName'] = info['info']['awayTeam']['name']
                    d_away_stats['formation'] = info['lineups']['away']['formation']
                    d_away_stats['isHome'] = False
                    d_away_stats['opponentId'] = info['info']['homeTeam']['id']

                    # Add score information if available
                    try:
                        d_home_stats['score'] = info['info']['homeScore']['display']
                        d_away_stats['score'] = info['info']['awayScore']['display']
                    except KeyError:
                        print(f"Score data missing for match {id}")
                        continue  # Skip this match if score data is missing

                    # Extract statistical data for both teams
                    for group in info['statistics']['statistics'][0]['groups']:
                        for item in group['statisticsItems']:
                            # Get individual statistics for each team
                            d_home_stats[item['name']] = item['homeValue']
                            d_away_stats[item['name']] = item['awayValue']

                            # If total values are available, add them
                            if 'homeTotal' in item.keys() and 'awayTotal' in item.keys():
                                d_home_stats[f"{item['name']} Total"] = item['homeTotal']
                                d_away_stats[f"{item['name']} Total"] = item['awayTotal']

                    # Append each team's stats to the list
                    l_match_stats.append(d_home_stats)
                    l_match_stats.append(d_away_stats)

Processing league: ENG


UnpicklingError: pickle data was truncated

In [ ]:
# Create the DataFrame from the list of match stats
df_match_stats = pd.DataFrame(df_match_stats)

# Set display option to show all rows
pd.set_option('display.max_rows', None)

# Display the entire DataFrame
df_match_stats

NameError: name 'l_match_stats' is not defined

In [ ]:
df_match_stats = pd.DataFrame(l_match_stats)

# Define the list of columns to keep for analysis
columns_of_interest = [
    'league', 'score', 'Expected goals', 'Ball possession', 'Total shots', 'Shots on target',
    'Shots off target', 'Blocked shots', 'Corner kicks', 'Offsides', 'Fouls', 'Yellow cards',
    'Free kicks', 'Throw-ins', 'Goal kicks', 'Big chances', 'Big chances missed',
    'Shots inside box', 'Shots outside box', 'Goalkeeper saves', 'Goals prevented',
    'Passes', 'Accurate passes', 'Long balls', 'Long balls Total', 'Crosses', 'Crosses Total',
    'Dribbles', 'Dribbles Total', 'Possession lost', 'Duels won', 'Aerials won', 'Tackles',
    'Interceptions', 'Clearances', 'Hit woodwork', 'Counter attacks', 'Counter attack shots',
    'Counter attack goals', 'Red cards'
]

# Filter the DataFrame to include only the columns of interest and replace NaN with 0
df_match_stats = df_match_stats[columns_of_interest].fillna(0)

# Group by league and calculate the mean for each statistic
df_league_averages = df_match_stats.groupby('league').mean().round(2).reset_index()

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set the width of the display
# Display the resulting DataFrame
print(df_league_averages)

  league  score  Expected goals  Ball possession  Total shots  Shots on target  Shots off target  Blocked shots  Corner kicks  Offsides  Fouls  Yellow cards  Free kicks  Throw-ins  Goal kicks  Big chances  Big chances missed  Shots inside box  Shots outside box  Goalkeeper saves  Goals prevented  Passes  Accurate passes  Long balls  Long balls Total  Crosses  Crosses Total  Dribbles  Dribbles Total  Possession lost  Duels won  Aerials won  Tackles  Interceptions  Clearances  Hit woodwork  Counter attacks  Counter attack shots  Counter attack goals  Red cards
0    ENG   1.43            1.45             50.0        12.64             4.41              4.67           3.56          5.04      1.70  10.78          1.84       10.34      18.98        7.20         2.07                1.22              8.44               4.20              2.93            -0.09  452.80           367.92       23.70             52.22     4.13          17.23      8.33           15.97           135.16      49.37      

In [ ]:
import pandas as pd
from collections import Counter

# Convert l_match_stats to a DataFrame
df_match_stats = pd.DataFrame(l_match_stats)

# Define columns of interest, excluding 'formation' for the mean calculations
numeric_columns = [
    'score', 'Expected goals', 'Ball possession', 'Total shots',
    'Shots on target', 'Shots off target', 'Blocked shots', 'Corner kicks',
    'Offsides', 'Fouls', 'Yellow cards', 'Free kicks', 'Throw-ins',
    'Goal kicks', 'Big chances', 'Big chances missed', 'Shots inside box',
    'Shots outside box', 'Goalkeeper saves', 'Goals prevented', 'Passes',
    'Accurate passes', 'Long balls', 'Long balls Total', 'Crosses',
    'Crosses Total', 'Dribbles', 'Dribbles Total', 'Possession lost',
    'Duels won', 'Aerials won', 'Tackles', 'Interceptions',
    'Clearances', 'Hit woodwork', 'Counter attacks',
    'Counter attack shots', 'Counter attack goals', 'Red cards'
]

# Filter the DataFrame to include only the numeric columns and the teamName
df_numeric_stats = df_match_stats[['league', 'teamName'] + numeric_columns].fillna(0)

# Group by league and teamName to calculate the mean for each club within each league
df_club_averages = df_numeric_stats.groupby(['league', 'teamName']).mean().round(2).reset_index()

# Calculate matches played for each team
matches_played = df_match_stats.groupby('teamName')['matchId'].nunique().reset_index()
matches_played.columns = ['teamName', 'matchesPlayed']

# Calculate most used formations for each team and their counts
formation_data = (
    df_match_stats.groupby(['teamName', 'formation']).size().reset_index(name='formationCount')
)
most_used_formations = (
    formation_data.loc[formation_data.groupby('teamName')['formationCount'].idxmax()]
    .reset_index(drop=True)
)

# Rename columns for clarity and prepare to merge
most_used_formations.columns = ['teamName', 'Formation', 'formationCount']

# Merge the matches played and most used formations with the club averages
df_club_averages = df_club_averages.merge(matches_played, on='teamName', how='left')
df_club_averages = df_club_averages.merge(most_used_formations, on='teamName', how='left')

# Rearranging columns: Move 'Formation' next to 'teamName'
df_club_averages = df_club_averages[['league', 'teamName', 'matchesPlayed', 'Formation', 'formationCount'] + numeric_columns]

# Create separate tables for each league
leagues = df_club_averages['league'].unique()  # Get list of leagues
league_tables = {}  # Dictionary to store each league's table

for league in leagues:
    # Filter by league and store the resulting table in the dictionary
    league_tables[league] = df_club_averages[df_club_averages['league'] == league]
    print(f"\nAverage stats for clubs in {league} league:")
    print(league_tables[league])

# Optional: Display all columns in Colab without wrapping for each table
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)



Average stats for clubs in ENG league:
   league        teamName  matchesPlayed Formation  formationCount  score  Expected goals  Ball possession  Total shots  Shots on target  Shots off target  Blocked shots  Corner kicks  Offsides  Fouls  Yellow cards  Free kicks  Throw-ins  Goal kicks  Big chances  Big chances missed  Shots inside box  Shots outside box  Goalkeeper saves  Goals prevented  Passes  Accurate passes  Long balls  Long balls Total  Crosses  Crosses Total  Dribbles  Dribbles Total  Possession lost  Duels won  Aerials won  Tackles  Interceptions  Clearances  Hit woodwork  Counter attacks  Counter attack shots  Counter attack goals  Red cards
0     ENG         Arsenal             38     4-3-3              38   2.32            1.94            59.76        15.61             5.37              5.39           4.84          5.87      1.47   9.82          1.37       11.45      16.18        4.87         2.61                1.32             10.89               4.71              2.50

In [ ]:
# Assuming l_match_stats is already defined and converted into a DataFrame
df_match_stats = pd.DataFrame(l_match_stats)

# Define a function to calculate the required metrics for each team
def calculate_metrics(df):
    metrics = {}

    # Calculate Defensive Actions
    defensive_actions = df['Tackles'].sum() + df['Interceptions'].sum() + df['Clearances'].sum()

    # Directness
    metrics['Directness'] = round(df['Total shots'].sum() / df['Ball possession'].sum(), 2) if df['Ball possession'].sum() != 0 else 0

    # Shot Distance
    metrics['Shot Distance'] = round(df['Shots outside box'].sum() / df['Total shots'].sum(), 2) if df['Total shots'].sum() != 0 else 0

    # Dribbles
    metrics['Dribbles'] = round(df['Dribbles Total'].sum() / df['Ball possession'].sum(), 2) if df['Ball possession'].sum() != 0 else 0

    # Crosses
    metrics['Crosses'] = round(df['Crosses Total'].sum() / df['Passes'].sum(), 2) if df['Passes'].sum() != 0 else 0

    # Long Balls
    metrics['Long Balls'] = round(df['Long balls Total'].sum() / df['Passes'].sum(), 2) if df['Passes'].sum() != 0 else 0

    # High Press
    metrics['High Press'] = round(df['Possession lost'].sum() / defensive_actions, 2) if defensive_actions != 0 else 0

    # Offside Trap
    metrics['Offside Trap'] = round(df['Offsides'].sum() / df['Possession lost'].sum(), 2) if df['Possession lost'].sum() != 0 else 0

    # Defender Intensity
    metrics['Defender Intensity'] = round(defensive_actions / df['Possession lost'].sum(), 2) if df['Possession lost'].sum() != 0 else 0

    # Fouls
    metrics['Fouls'] = round(df['Fouls'].sum() / defensive_actions, 2) if defensive_actions != 0 else 0

    # Counter-Attack
    metrics['Counter-Attack'] = round(df['Counter attack shots'].sum() / df['Total shots'].sum(), 2) if df['Total shots'].sum() != 0 else 0

    # Chance Creation
    metrics['Chance Creation'] = round(df['Expected goals'].sum(), 2)

    # Finishing
    metrics['Finishing'] = round(df['score'].sum() - df['Expected goals'].sum(), 2)

    # Chance Prevention
    metrics['Chance Prevention'] = round(1 - (df['score'].sum() - df['Expected goals'].sum()), 2)

    # Sloppiness
    metrics['Sloppiness'] = round(df['Possession lost'].sum() / df['Ball possession'].sum(), 2) if df['Ball possession'].sum() != 0 else 0

    return metrics

# Calculate metrics for each team in each league
df_metrics = df_match_stats.groupby(['league', 'teamName']).apply(calculate_metrics).reset_index()

# Create separate DataFrames for each league
leagues = df_metrics['league'].unique()
league_tables = {}

for league in leagues:
    league_table = df_metrics[df_metrics['league'] == league]
    league_tables[league] = league_table.reset_index(drop=True)
    print(f"\nMetrics for teams in {league} league:")
    print(league_tables[league])

# Optional: Display all columns in Colab without wrapping for each table
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


Metrics for teams in ENG league:
   league        teamName                                                  0
0     ENG         Arsenal  {'Directness': 0.26, 'Shot Distance': 0.3, 'Dr...
1     ENG     Aston Villa  {'Directness': 0.23, 'Shot Distance': 0.35, 'D...
2     ENG     Bournemouth  {'Directness': 0.23, 'Shot Distance': 0.28, 'D...
3     ENG       Brentford  {'Directness': 0.25, 'Shot Distance': 0.27, 'D...
4     ENG        Brighton  {'Directness': 0.27, 'Shot Distance': 0.33, 'D...
5     ENG         Chelsea  {'Directness': 0.22, 'Shot Distance': 0.33, 'D...
6     ENG  Crystal Palace  {'Directness': 0.24, 'Shot Distance': 0.37, 'D...
7     ENG         Everton  {'Directness': 0.27, 'Shot Distance': 0.35, 'D...
8     ENG          Forest  {'Directness': 0.26, 'Shot Distance': 0.35, 'D...
9     ENG          Fulham  {'Directness': 0.23, 'Shot Distance': 0.34, 'D...
10    ENG           Leeds  {'Directness': 0.26, 'Shot Distance': 0.32, 'D...
11    ENG       Leicester  {'Directness': 

In [ ]:
# Assuming df_metrics is already defined and has the metrics in the proper format
def flatten_metrics(df):
    # Create a new DataFrame for each team's metrics
    metrics_df = pd.json_normalize(df[0])  # Expanding the metrics dictionary
    metrics_df['league'] = df['league']
    metrics_df['teamName'] = df['teamName']
    return metrics_df

# Flatten the metrics for each league and combine them into a single DataFrame
flattened_metrics = pd.concat(df_metrics.apply(flatten_metrics, axis=1).tolist(), ignore_index=True)

# Format all float values to two decimal points
flattened_metrics = flattened_metrics.round(2)

# Display metrics for each league
leagues = flattened_metrics['league'].unique()
league_tables = {}

for league in leagues:
    league_table = flattened_metrics[flattened_metrics['league'] == league]
    league_tables[league] = league_table.reset_index(drop=True)
    print(f"\nAverage stats for clubs in {league} league:")
    print(league_tables[league])

# Optional: Display all columns in Colab without wrapping for each table
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


Average stats for clubs in ENG league:
    Directness  Shot Distance  Dribbles  Crosses  Long Balls  High Press  Offside Trap  Defender Intensity  Fouls  Counter-Attack  Chance Creation  Finishing  Chance Prevention  Sloppiness league        teamName
0         0.26           0.30      0.28     0.03        0.08        3.59          0.01                0.28   0.27            0.02            73.64      14.36             -13.36        2.17    ENG         Arsenal
1         0.23           0.35      0.31     0.04        0.12        3.00          0.01                0.33   0.26            0.03            51.04      -0.04               1.04        2.59    ENG     Aston Villa
2         0.23           0.28      0.44     0.04        0.14        2.58          0.01                0.39   0.20            0.03            39.48      -2.48               3.48        3.29    ENG     Bournemouth
3         0.25           0.27      0.31     0.05        0.17        3.17          0.01                0.32   0.2

In [ ]:
# Assuming l_match_stats is already defined and converted into a DataFrame
df_match_stats = pd.DataFrame(l_match_stats)

# Define the list of columns to keep for analysis
columns_of_interest = [
    'league', 'score', 'Expected goals', 'Ball possession', 'Total shots', 'Shots on target',
    'Shots off target', 'Blocked shots', 'Corner kicks', 'Offsides', 'Fouls', 'Yellow cards',
    'Free kicks', 'Throw-ins', 'Goal kicks', 'Big chances', 'Big chances missed',
    'Shots inside box', 'Shots outside box', 'Goalkeeper saves', 'Goals prevented',
    'Passes', 'Accurate passes', 'Long balls', 'Long balls Total', 'Crosses', 'Crosses Total',
    'Dribbles', 'Dribbles Total', 'Possession lost', 'Duels won', 'Aerials won', 'Tackles',
    'Interceptions', 'Clearances', 'Hit woodwork', 'Counter attacks', 'Counter attack shots',
    'Counter attack goals', 'Red cards'
]

# Filter the DataFrame to include only the columns of interest and replace NaN with 0
df_match_stats = df_match_stats[columns_of_interest].fillna(0)

# Group by league and calculate the mean for each statistic
df_league_averages = df_match_stats.groupby('league').mean().round(2).reset_index()

# Define a function to calculate league-wide metrics
def calculate_league_metrics(df):
    metrics = {}

    # Calculate Defensive Actions
    defensive_actions = df['Tackles'].sum() + df['Interceptions'].sum() + df['Clearances'].sum()

    # Calculate metrics
    metrics['Directness'] = round(df['Total shots'].sum() / df['Ball possession'].sum(), 2) if df['Ball possession'].sum() != 0 else 0
    metrics['Shot Distance'] = round(df['Shots outside box'].sum() / df['Total shots'].sum(), 2) if df['Total shots'].sum() != 0 else 0
    metrics['Dribbles'] = round(df['Dribbles Total'].sum() / df['Ball possession'].sum(), 2) if df['Ball possession'].sum() != 0 else 0
    metrics['Crosses'] = round(df['Crosses Total'].sum() / df['Passes'].sum(), 2) if df['Passes'].sum() != 0 else 0
    metrics['Long Balls'] = round(df['Long balls Total'].sum() / df['Passes'].sum(), 2) if df['Passes'].sum() != 0 else 0
    metrics['High Press'] = round(df['Possession lost'].sum() / defensive_actions, 2) if defensive_actions != 0 else 0
    metrics['Offside Trap'] = round(df['Offsides'].sum() / df['Possession lost'].sum(), 2) if df['Possession lost'].sum() != 0 else 0
    metrics['Defender Intensity'] = round(defensive_actions / df['Possession lost'].sum(), 2) if df['Possession lost'].sum() != 0 else 0
    metrics['Fouls'] = round(df['Fouls'].sum() / defensive_actions, 2) if defensive_actions != 0 else 0
    metrics['Counter-Attack'] = round(df['Counter attack shots'].sum() / df['Total shots'].sum(), 2) if df['Total shots'].sum() != 0 else 0
    metrics['Chance Creation'] = round(df['Expected goals'].sum(), 2)
    metrics['Finishing'] = round(df['score'].sum() - df['Expected goals'].sum(), 2)
    metrics['Chance Prevention'] = round(1 - (df['score'].sum() - df['Expected goals'].sum()), 2)
    metrics['Sloppiness'] = round(df['Possession lost'].sum() / df['Ball possession'].sum(), 2) if df['Ball possession'].sum() != 0 else 0

    return metrics

# Calculate metrics for each league only
league_metrics = df_match_stats.groupby('league').apply(calculate_league_metrics).reset_index()

# Create a DataFrame from the league metrics
league_kpis_df = pd.DataFrame(league_metrics[0].tolist())
league_kpis_df['league'] = league_metrics['league']

# Rearranging columns to have 'league' first
league_kpis_df = league_kpis_df[['league'] + [col for col in league_kpis_df.columns if col != 'league']]

# Display the league-wide KPIs in the desired format
print("\nLeague-wide KPIs in desired format:")
print(league_kpis_df)

# Optionally display average metrics by league
print("\nAverage metrics by league:")
print(df_league_averages)


League-wide KPIs in desired format:
  league  Directness  Shot Distance  Dribbles  Crosses  Long Balls  High Press  Offside Trap  Defender Intensity  Fouls  Counter-Attack  Chance Creation  Finishing  Chance Prevention  Sloppiness
0    ENG        0.25           0.33      0.32     0.04        0.12        3.04          0.01                0.33   0.24            0.04          1100.81     -16.81              17.81        2.70
1    FRA        0.25           0.36      0.36     0.04        0.11        2.95          0.01                0.34   0.28            0.04          1083.22     -16.22              17.22        2.61
2    GER        0.25           0.34      0.32     0.04        0.14        3.31          0.01                0.30   0.27            0.04           896.80      81.20             -80.20        2.92
3    ITA        0.25           0.37      0.30     0.04        0.12        3.23          0.01                0.31   0.30            0.04           976.41       1.59              -0.59 

In [ ]:
merged_df = pd.merge(league_kpis_df, df_league_averages, on='league', how='inner')

# Display the merged DataFrame
print("Merged DataFrame of League KPIs and Average Metrics:")
print(merged_df)

Merged DataFrame of League KPIs and Average Metrics:
  league  Directness  Shot Distance  Dribbles_x  Crosses_x  Long Balls  High Press  Offside Trap  Defender Intensity  Fouls_x  Counter-Attack  Chance Creation  Finishing  Chance Prevention  Sloppiness  score  Expected goals  Ball possession  Total shots  Shots on target  Shots off target  Blocked shots  Corner kicks  Offsides  Fouls_y  Yellow cards  Free kicks  Throw-ins  Goal kicks  Big chances  Big chances missed  Shots inside box  Shots outside box  Goalkeeper saves  Goals prevented  Passes  Accurate passes  Long balls  Long balls Total  Crosses_y  Crosses Total  Dribbles_y  Dribbles Total  Possession lost  Duels won  Aerials won  Tackles  Interceptions  Clearances  Hit woodwork  Counter attacks  Counter attack shots  Counter attack goals  Red cards
0    ENG        0.25           0.33        0.32       0.04        0.12        3.04          0.01                0.33     0.24            0.04          1100.81     -16.81              1